# `VKS document with llama3`


In [1]:
%load_ext autoreload
%autoreload 2

## On its own


In [2]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PDFMinerToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter

/home/cuongdm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
raw_document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

pipeline = Pipeline()
pipeline.add_component("converter", PDFMinerToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=20, split_overlap=15))
pipeline.add_component("writer", DocumentWriter(document_store=raw_document_store))

pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "writer")

🚅 Components
  - converter: PDFMinerToDocument
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - writer: DocumentWriter
🛤️ Connections
  - converter.documents -> cleaner.documents (List[Document])
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> writer.documents (List[Document])

In [4]:
file_names = ["vi-vks.pdf"]
pipeline.run({"converter": {"sources": file_names}})

{'writer': {'documents_written': 769}}

In [5]:
for doc in raw_document_store.filter_documents():
    print(doc.content[:500])

VKS
VKS  VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn
đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là
một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý
và triển khai các ứng dụng container trên môi trường phân tán.
VKS Demo Video - Giải Pháp Quản Lý Kubernetes Toàn Diện Của VNG Cloud
VKS Demo Video - Giải Pháp Quản Lý Kubernetes Toàn Diện Của VNG Cloud
2VKS là gì?
VKS  VNGCloud Kub

3• Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản
Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29  để đảm bảo bạn luôn tận dụng
được những tính năng tiên tiến nhất.
• Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật
cao.
• Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ
dàng và nhanh chóng, giúp bạn luôn cập nhật những cải tiến mới nhất.
• Scaling & Healing Automatically: VKS tự động mở rộng

In [6]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [7]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

Initalize a Document Embedder

In [8]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [9]:
docs_with_embeddings = doc_embedder.run(raw_document_store.filter_documents())
document_store.write_documents(docs_with_embeddings["documents"])

Batches: 100%|██████████| 25/25 [00:24<00:00,  1.03it/s]


769

In [10]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

In [11]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
template = [ChatMessage.from_user("""
Cung cấp các thông tin sau, trả các câu hỏi.

Thông tin:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Câu hỏi: {{question}}
Trả lời:
""")]

prompt_builder = ChatPromptBuilder(template=template)

In [13]:
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://61.28.230.60:11434",
    generation_kwargs={
        "num_predict": 1024,
        "temperature": 0.01})

Initialize retriever

In [14]:
retriever = InMemoryEmbeddingRetriever(document_store)

In [15]:
basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", chat_generator)

In [16]:
# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

# Asking question

In [17]:
question = "VKS là gì?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 90.64it/s]


VKS (VNG Cloud Kubernetes Service) là dịch vụ quản lý container và ứng dụng trên nền tảng Kubernetes do VNG Cloud cung cấp.

In [18]:
question = "So sánh VKS private cluster và public cluster"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 83.08it/s]


VKS Private Cluster và Public Cluster là hai mô hình khác nhau trong việc quản lý và triển khai các ứng dụng trên nền tảng Kubernetes (K8S) của VNG Cloud. Dưới đây là một số điểm khác biệt chính giữa hai mô hình này:

**1. Bảo mật**

* VKS Private Cluster: Mọi kết nối hoàn toàn là private từ kết nối giữa nodes tới control plane, kết nối từ client tới control plane, hay kết nối từ nodes tới các sản phẩm dịch vụ khác trong VNG Cloud như vStorage, vCR, vMonitor, VNGCloud APIs,...
* VKS Public Cluster: Các public cluster trên VKS đang sử dụng địa chỉ Public IP để giao tiếp giữa nodes và control plane.

**2. Chi phí**

* VKS Private Cluster: Việc sử dụng private cluster sẽ phát sinh thêm chi phí cho 4 private service endpoint.
* VKS Public Cluster: Không có chi phí bổ sung nào liên quan đến bảo mật.

**3. Tính linh hoạt**

* VKS Private Cluster: Các node trong Cluster khi được tạo ra sẽ chỉ có internal IP và không thể đi ra public internet. Nếu muốn node truy cập được internet, bạn cần sử d

In [19]:
question = "Làm sao để sử dụng vngcloud blockstorage csi driver"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 78.75it/s]


Để sử dụng vngcloud blockstorage csi driver, bạn cần thực hiện các bước sau:

1. Cài đặt Helm phiên bản từ 3.0 trở lên.
2. Thêm repo này vào cluster của bạn qua lệnh:
```bash
helm repo add vks-helm-charts https://vngcloud.github.io/vks-helm-chart
```
3. Cập nhật repo:
```bash
helm repo update
```
4. Lấy ClientID và Secret_Key của Service Account đã được tạo trước đó.
5. Thay thế thông tin ClientID, Client Secret và ClusterID của cụm K8S của bạn vào lệnh sau:
```bash
helm install vngcloud-blockstorage-csi-driver vks-helm-charts/vngcloud-blockstorage-csi-driver --namespace kube-system \
  --set cloudConfig.global.clientID=<Lấy ClientID của Service Account> \
  --set cloudConfig.global.clientSecret=<Lấy Client Secret của Service Account> \
  --set cluster.id=<Cluster ID của cụm K8S>
```
6. Sau khi cài đặt thành công, bạn có thể sử dụng vngcloud blockstorage csi driver bằng cách thêm nó vào file `values.yaml` hoặc tạo một file mới với nội dung:
```yaml
csi:
  drivers:
    - name: vngcloud-

In [20]:
question = "Làm sao để tạo một pvc 100Gi"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.78it/s]


Để tạo một Persistent Volume Claim (PVC) với dung lượng 100 Gi, bạn cần thực hiện các bước sau:

1. Tạo một file YAML chứa thông tin về PVC:
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-100gi
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 100Gi
```
2. Tạo PVC bằng cách áp dụng file YAML này vào Kubernetes cluster của bạn:
```bash
kubectl apply -f pvc-100gi.yaml
```
3. Kiểm tra trạng thái của PVC:
```bash
kubectl get pvc pvc-100gi
```
Nếu PVC được tạo thành công, bạn sẽ thấy thông tin về nó trong kết quả đầu ra.

Lưu ý: Trước khi tạo PVC, bạn cần đảm bảo rằng cluster Kubernetes của mình đã được cấu hình để sử dụng Persistent Volumes (PV) và có đủ dung lượng lưu trữ để đáp ứng yêu cầu của PVC.

In [21]:
question = "Sử dụng snapshot trong vks csi như thế nào"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.87it/s]


Để sử dụng snapshot trong vKS CSI (Container Storage Interface), bạn cần thực hiện các bước sau:

1. **Tạo một ví POC**: Trước khi tạo snapshot, bạn cần tạo một ví POC (Proof of Concept) để có thể sử dụng dịch vụ này.
2. **Chọn Activate Snapshot**: Tại màn hình vServer Portal, chọn Activate Snapshot.
3. **Chọn sử dụng ví POC**: Tại màn hình Checkout, chọn sử dụng ví POC để tạo snapshot.

Sau khi thực hiện các bước trên, tất cả các resource snapshot của bạn sẽ được tạo qua ví POC. Do đó, việc stop POC cần được bạn thực hiện thông qua vConsole hoặc vServer Portal.

Lưu ý: Việc sử dụng snapshot trong vKS CSI giúp bạn có thể dễ dàng tạo và quản lý các bản sao lưu của dữ liệu ứng dụng, đảm bảo tính sẵn sàng và phục hồi dữ liệu khi cần thiết.

In [22]:
question = "CSI của vngcloud có hỗ trợ change volume size không"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.59it/s]


CSI (Container Storage Interface) của VNG Cloud hỗ trợ thay đổi kích thước volume (change volume size). Bạn có thể thực hiện việc này bằng cách sử dụng lệnh `kubectl patch` để cập nhật thông tin về kích thước volume trong tệp YAML của Persistent Volume Claim (PVC).

Ví dụ, nếu bạn muốn tăng kích thước volume từ 20 Gi lên 30 Gi, bạn có thể chạy lệnh sau:

```
kubectl patch pvc my-expansion-pvc -p '{"spec":{"resources":{"requests":{"storage":"30Gi"}}}}'
```

Sau khi thực hiện lệnh trên, bạn có thể kiểm tra lại thông tin về PVC để đảm bảo rằng kích thước volume đã được cập nhật thành công.

Lưu ý: Việc thay đổi kích thước volume chỉ hỗ trợ tăng lên mà không hỗ trợ giảm xuống. Nếu bạn muốn giảm kích thước volume, bạn cần phải xóa dữ liệu trong volume trước khi thực hiện việc này.